In [1]:
#coding:utf-8

import pandas as pd

from xtquant import xtdata

In [5]:
# 下载002594的日K线数据
print('开始下载002594的日K线数据')
xtdata.download_history_data(stock_code='002594.SZ', period='1d', incrementally=True)
print('下载002594的日K线数据完成')


开始下载002594的日K线数据
下载002594的日K线数据完成


In [6]:
history_data = xtdata.get_market_data_ex([], ['002594.SZ'], period='1d')
print(f'获取到{len(history_data)}条历史数据，数据如下：')
print(history_data)

获取到1条历史数据，数据如下：
{'002594.SZ':                    time    open    high     low   close  volume        amount  \
20110630  1309363200000   22.00   26.19   22.00   25.45  562925  1.349342e+09   
20110701  1309449600000   25.77   28.00   25.00   28.00  335612  8.986792e+08   
20110704  1309708800000   28.88   30.80   28.58   30.80  219645  6.535179e+08   
20110705  1309795200000   32.78   33.88   31.89   33.05  381080  1.257046e+09   
20110706  1309881600000   32.80   35.55   32.11   33.40  312967  1.056614e+09   
...                 ...     ...     ...     ...     ...     ...           ...   
20250421  1745164800000  346.00  349.87  345.24  348.76   74953  2.609226e+09   
20250422  1745251200000  348.76  356.93  346.78  354.84  112122  3.956086e+09   
20250423  1745337600000  377.00  378.00  367.23  371.99  247927  9.216822e+09   
20250424  1745424000000  372.48  374.60  363.35  366.00  150892  5.557113e+09   
20250425  1745510400000  370.00  375.30  368.10  370.83  133869  4.975315e+09  

In [7]:
def gen_divid_ratio(quote_datas, divid_datas):
    drl = []
    dr = 1.0
    qi = 0
    qdl = len(quote_datas)
    di = 0
    ddl = len(divid_datas)
    while qi < qdl and di < ddl:
        qd = quote_datas.iloc[qi]
        dd = divid_datas.iloc[di]
        if qd.name >= dd.name:
            dr *= dd['dr']
            di += 1
        if qd.name <= dd.name:
            drl.append(dr)
            qi += 1
    while qi < qdl:
        drl.append(dr)
        qi += 1
    return pd.DataFrame(drl, index = quote_datas.index, columns = quote_datas.columns)

def process_forward_ratio(quote_datas, divid_datas):
    drl = gen_divid_ratio(quote_datas, divid_datas)
    drlf = drl / drl.iloc[-1]
    result = (quote_datas * drlf).apply(lambda x: round(x, 2))
    return result

def process_backward_ratio(quote_datas, divid_datas):
    drl = gen_divid_ratio(quote_datas, divid_datas)
    result = (quote_datas * drl).apply(lambda x: round(x, 2))
    return result

def process_forward(quote_datas1, divid_datas):
    quote_datas = quote_datas1.copy()
    def calc_front(v, d):
        return ((v - d['interest'] + d['allotPrice'] * d['allotNum'])
            / (1 + d['allotNum'] + d['stockBonus'] + d['stockGift']))
    for qi in range(len(quote_datas)):
        q = quote_datas.iloc[qi]
        for di in range(len(divid_datas)):
            d = divid_datas.iloc[di]
            if d.name <= q.name:
                continue
            q.iloc[0] = calc_front(q.iloc[0], d)
    return quote_datas

def process_backward(quote_datas1, divid_datas):
    quote_datas = quote_datas1.copy()
    def calc_back(v, d):
        return ((v * (1.0 + d['stockGift'] + d['stockBonus'] + d['allotNum'])
            + d['interest'] - d['allotNum'] * d['allotPrice']))
    for qi in range(len(quote_datas)):
        q = quote_datas.iloc[qi]
        for di in range(len(divid_datas) - 1, -1, -1):
            d = divid_datas.iloc[di]
            if d.name > q.name:
                continue
            q.iloc[0] = calc_back(q.iloc[0], d)
    return quote_datas


#--------------------------------

s = '002594.SZ'

#xtdata.download_history_data(s, '1d', '20100101', '')
# 获取除权除息日及对应的权息
dd = xtdata.get_divid_factors(s)
print(f'获取到{len(dd)}条除权除息日数据，数据如下：')
print(dd)

#复权计算用于处理价格字段
field_list = ['open', 'high', 'low', 'close']

stock_price002594 = xtdata.get_market_data(field_list, [s], '1d', dividend_type = 'none')
print(f'获取到002594的{len(stock_price002594)}条价格数据，数据如下：')
print(stock_price002594)
datas_ori = stock_price002594['close'].T
# 转置后的数据
print(f'获取到{len(datas_ori)}条收盘价，数据如下：')
print(datas_ori)

#等比前复权
datas_forward_ratio = process_forward_ratio(datas_ori, dd)
print('datas_forward_ratio', datas_forward_ratio)

#等比后复权
datas_backward_ratio = process_backward_ratio(datas_ori, dd)
print('datas_backward_ratio', datas_backward_ratio)

#前复权
datas_forward = process_forward(datas_ori, dd)
print('datas_forward', datas_forward)

#后复权
datas_backward = process_backward(datas_ori, dd)
print('datas_backward', datas_backward)



获取到10条除权除息日数据，数据如下：
                  time  interest  stockBonus  stockGift  allotNum  allotPrice  \
20140813  1.407859e+12  0.050000         0.0        0.0       0.0         0.0   
20161213  1.481558e+12  0.367000         0.0        0.0       0.0         0.0   
20170804  1.501776e+12  0.178000         0.0        0.0       0.0         0.0   
20180817  1.534435e+12  0.141000         0.0        0.0       0.0         0.0   
20190726  1.564070e+12  0.204000         0.0        0.0       0.0         0.0   
20200818  1.597680e+12  0.060000         0.0        0.0       0.0         0.0   
20210806  1.628179e+12  0.148000         0.0        0.0       0.0         0.0   
20220729  1.659024e+12  0.105000         0.0        0.0       0.0         0.0   
20230728  1.690474e+12  1.142000         0.0        0.0       0.0         0.0   
20240729  1.722182e+12  3.097772         0.0        0.0       0.0         0.0   

          gugai        dr  
20140813    0.0  1.000970  
20161213    0.0  1.007482  
2017